In [1]:
__version__='0.1.2'
# env mar.2020
# Import the required libraries
import bql
import bqport
import numpy as np
import pandas as pd
import heatMap
from collections import OrderedDict
#from bqwidgets import TickerAutoComplete, DataGrid, DatePicker
import bqwidgets as bqw
import bqviz as bqv
import bqplot as bqp
from bqplot import  Tooltip,Figure,Lines, Axis
from bqplot.interacts import IndexSelector
from ipywidgets import *
import bqwidgets as bqw
from IPython.display import display, clear_output
np.warnings.filterwarnings('ignore', category=np.RankWarning)
import datetime as dt
import warnings
from bql.om.exceptions import InvalidParameterGroupError
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 
warnings.filterwarnings("ignore") 
#Loading animation
loading_html = HTML("""
    <div style="font-size:14px; color:lightskyblue;">
        <i class="fa fa-circle-o-notch fa-spin"></i><span>&nbsp;Loading...</span>
    </div>""")

preload_box = HBox([loading_html])
preload_box

In [2]:
# Create an instance of BQL for querying data.
# Instantiate an object to interface with the BQL service
if "bq" not in globals(): 
    bq = bql.Service()


In [3]:
path=''
color_selection = '#F39F41'

import pickle
from collections import OrderedDict
from datetime import timedelta

try: 
    f = open(path + 'settings_breadth.pckl', 'rb')
    deft_settings = pickle.load(f)
    f.close() 
    
except:
    deft_settings = OrderedDict()
    deft_settings['universe_settings'] = {}
    deft_settings['date_settings'] = {}
    
def save_settings(caller=None):
    dict_settings = OrderedDict()
    dict_settings['universe_settings'] = universe_picker.settings
    dict_settings['date_settings'] = date_picker.settings
    
    f=open(path + 'settings_breadth.pckl','wb')
    pickle.dump(dict_settings, f)
    f.close()

In [4]:
import apputils

bq = bql.Service()
#apputils.bq = bq

In [5]:
class DatePicker:
    def __init__(self, deft_settings=None):
        try:
            defalut_interval = deft_settings['univ_dropdown']
        except:
            defalut_interval = '365d'
        self._label_as_of_date = Label(value='As of Date:',layout=Layout(width='180px'))
        self.as_of_date_picker=bqw.DatePicker(value=(dt.datetime.now()).strftime('%Y-%m-%d'),date_format='%Y-%m-%d')
        self._label_interval = Label(value='History Length (days)',layout=Layout(width='180px'))
        self.text_interval = Text(value=defalut_interval)
        self._box = VBox([HBox([self._label_interval,self.text_interval]), 
                          HBox([self._label_as_of_date,self.as_of_date_picker])])
    def show(self):
        return self._box
    
    @property
    def settings(self):
        dict_settings={}
        dict_settings['interval'] = self._label_interval.value
        return dict_settings

In [6]:
universe_picker = apputils.UniversePicker(lst_universe_types=['Index', 'Portfolio', 'Exchanges', 'List', 'All Active Equities'],ipw_container=False,deft_settings=deft_settings['universe_settings'],
                                         max_universe_count=20000, _bq=bq)
date_picker = DatePicker(deft_settings=deft_settings['date_settings'])

In [7]:
# Senting
upper_thurshold={}
upper_thurshold['default'] = 50
upper_thurshold['Advanced'] = 60
upper_thurshold['Declined'] = 60
upper_thurshold['52w High'] = 30
upper_thurshold['52w Low'] = 30
upper_thurshold['4w High'] = 50
upper_thurshold['4w Low'] = 50
upper_thurshold['RSI < 30'] = 20
upper_thurshold['RSI > 70'] = 20
upper_thurshold['>10D Avg'] = 80
upper_thurshold['>50D Avg'] = 60
upper_thurshold['>250D Avg'] = 60
upper_thurshold['Trender'] = 60
upper_thurshold['>Boll UB'] = 30
upper_thurshold['&ltBoll LB'] = 30

lower_thurshold={}
lower_thurshold['>10D Avg'] = 20
lower_thurshold['>50D Avg'] = 40
lower_thurshold['>250D Avg'] = 40
lower_thurshold['Trender'] = 40

lower_is_better_fields = ['Declined','52w Low','RSI < 30','4w Low','52w Low','&ltBoll LB']

In [8]:
def _get_sector_details():
    #global df_details, request, response
    as_of_date=asof_datepicker.value
    label_as_of = Label('As of: ' + date_picker.as_of_date_picker.value)
    datafields=OrderedDict()
    datafields['Name'] = bq.data.name().value(mapby="ORDINAL")
    px_last = bq.data.px_last(start='-10d',end=as_of_date,per='d',ca_adj='full')
    dates = px_last['DATE']
    for k,v in build_bq_dateitem([bq.data.id(),dates],'-10d',as_of_date).items():
        datafields[k] = v
    #datafields.update(build_bq_dateitem(bq.data.id(),'-10d',as_of_date))
    request =  bql.Request(bq.univ.filter(universe_picker.universe,universe_picker.grouping_bqitem==selected_sector), datafields)
    response = bq.execute(request)
    rename_col = {dates.format_for_query_string().upper():'Date',bq.data.id().format_for_query_string().upper():'ID'}
    def merge(response): return pd.concat([sir.df().rename(columns=rename_col).set_index(['ID','Date']).dropna(how='all')[sir.name] for sir in response[1:]], axis=1)
    #def merge(response): return pd.concat([sir.df()[sir.name] for sir in response], axis=1)
    df_details = merge(response)
    df_details = df_details.reset_index()
    df_details = df_details.loc[df_details['Date'] == df_details['Date'].max()].set_index('ID')
    df_details = response[0].df().join(df_details)
    df_details.index.name='ID'
    df_details = df_details.reset_index()
    df_details['ID'] = df_details['ID'].apply(lambda x:x.split(' ')[0] + ': ')
    df_details['ID'] = df_details['ID'] + df_details['Name']
    df_details = df_details.set_index('ID')
    heatmap_details = heatMap.GenerateHeatMap()
#     heatmap_details.generate_heatmaps([{'data': df_details[['Name']],'table_mode':True,'width':'200px'}]+
#                           [{'data':df_details[[name]],'reverse_color':True if name in lower_is_better_fields else False,'upper_bound':1} for name in df_details.columns if name not in ['Name','#Company','Price']], 
#                       empty_space_width='0px',button_width='68px', header_height='30px',row_select=True, upper_bound=thurshold.get('default',None))
    heatmap_details.generate_heatmaps([{'data':df_details[[name]],'reverse_color':True if name in lower_is_better_fields else False,
                                        'upper_bound':1,
                                        'lower_bound':0 if lower_thurshold.get(name,None) else None
                                       } for name in df_details.columns if name not in ['Name','#Company','Price','Date','ORIG_IDS']], 
                      empty_space_width='0px',button_width='68px', header_height='30px',row_select=True, upper_bound=upper_thurshold.get('default',None))
    box_heatmap_details.children = [VBox([label_as_of,VBox(layout={'min_height':'20px'}), heatmap_details.get_widget()],layout={'overflow_y':'scroll'})]

In [9]:
def build_bq_dateitem(grouping, date_interval, as_of_date):

    px_last = bq.data.px_last(start=date_interval,end=as_of_date,per='d',ca_adj='full')
    px_last_w = bq.data.px_last(start=date_interval,end=as_of_date,per='w',ca_adj='full')
    px_high = bq.data.px_high(start=date_interval,end=as_of_date,per='d',ca_adj='full')
    px_low = bq.data.px_low(start=date_interval,end=as_of_date,per='d',ca_adj='full')
    dates = px_last['DATE']
    roc = bq.data.roc(px_last,period=1)
    mxmn = bq.data.maxmin(px_high,px_low)
    boll = bq.data.boll(px_last)

    datafields=OrderedDict()
    datafields['Advanced'] = (bq.func.if_(roc>0,1,0)).group(grouping).sum()/roc.group(grouping).count()
    datafields['Declined'] = (bq.func.if_(roc<0,1,0)).group(grouping).sum()/roc.group(grouping).count()
    datafields['>10D Avg'] = (bq.func.if_(px_last-bq.data.smavg(px_last,period=10)>0,1,0).group(grouping).sum()/
                px_last.group(grouping).count())
    datafields['>50D Avg'] = (bq.func.if_(px_last-bq.data.smavg(px_last,period=50)>0,1,0).group(grouping).sum()/
                px_last.group(grouping).count())
    datafields['>250D Avg'] = (bq.func.if_(px_last-bq.data.smavg(px_last,period=250)>0,1,0).group(grouping).sum()/
                px_last.group(grouping).count())
    datafields['4w High'] = (bq.func.if_(mxmn.with_additional_parameters(period=20)['maxline']/px_high==1,1,0).group(grouping).sum()/
                px_last.group(grouping).count())
    datafields['4w Low'] = (bq.func.if_(mxmn.with_additional_parameters(period=20)['minline']/px_low==1,1,0).group(grouping).sum()/
                px_last.group(grouping).count())
    datafields['52w High'] = (bq.func.if_(mxmn.with_additional_parameters(period=260)['maxline']/px_high==1,1,0).group(grouping).sum()/
                px_last.group(grouping).count())
    datafields['52w Low'] = (bq.func.if_(mxmn.with_additional_parameters(period=260)['minline']/px_low==1,1,0).group(grouping).sum()/
                px_last.group(grouping).count())
    datafields['RSI < 30'] = (bq.func.if_(bq.data.RSI(px_last,period=14)<30,1,0).group(grouping).sum()/
                px_last.group(grouping).count())
    datafields['RSI > 70'] = (bq.func.if_(bq.data.RSI(px_last,period=14)>70,1,0).group(grouping).sum()/
                px_last.group(grouping).count())
    
#     datafields['Abv BOLL'] = (bq.func.if_(boll['UPPERBAND']<px_last,1,0).group(grouping).sum()/
#                 px_last.group(grouping).count())
#     datafields['Blw BOLL'] = (bq.func.if_(boll['LOWERBAND']>px_last,1,0).group(grouping).sum()/
#                 px_last.group(grouping).count())
    datafields['MACD'] = (bq.func.if_(bq.data.macd(px_last)['MACD2']>0,1,0).group(grouping).sum()/
                              px_last.group(grouping).count())
    datafields['>Boll UB'] = (bq.func.if_(bq.data.boll(px_last)['UpperBand']<=px_last,1,0).group(grouping).sum()/
                              px_last.group(grouping).count())
    datafields['&ltBoll LB'] = (bq.func.if_(bq.data.boll(px_last)['LowerBand']>=px_last,1,0).group(grouping).sum()/
                              px_last.group(grouping).count())
#     datafields['MACD (W)'] = (bq.func.if_(bq.data.macd(px_last_w)['MACD2']>0,1,0).group(grouping).sum()/
#                               px_last_w.group(grouping).count())
    datafields['Trender'] = (bq.data.TRENDER(px_high, px_low, px_last,SENSITIVITY = 'MEDIUMTERM6')['TRENDERUP'].group(grouping).count()/
                px_last.group(grouping).count())
    return datafields



def button_run_on_click(btn=None):
    global grid_data, df_main, heatmap, lower_is_better_fields, selected_sector, box_heatmap_details, df_overall
    global RSI_Line_plot, adv_dec_plot
    global intsel, asof_datepicker, df_return, df_main, df_adv_dec, df_RSI, df_MA
    
    #box_status.children=[loading_html]
    as_of_date=date_picker.as_of_date_picker.value
    date_interval = date_picker.text_interval.value
    if date_interval[0]!='-': date_interval='-' + date_interval
    if date_interval[-1].upper()!='D': date_interval= date_interval +'D'
    
    px_last = bq.data.px_last(start=date_interval,end=as_of_date,per='d',ca_adj='full')
    px_high = bq.data.px_high(start=date_interval,end=as_of_date,per='d',ca_adj='full')
    px_low = bq.data.px_low(start=date_interval,end=as_of_date,per='d',ca_adj='full')
    mxmn = bq.data.maxmin(px_high,px_low)
    dates = px_last['DATE']
    sector = universe_picker.grouping_bqitem
    grouping = [sector, dates]
    datafields = build_bq_dateitem(grouping, date_interval, as_of_date)
    datafields['Mkt Cap'] = (bq.data.cur_mkt_cap(start=date_interval,end=as_of_date,per='d',currency='USD',fill='prev')/1000000).group(grouping).sum()
    datafields['Return'] = (bq.data.day_to_day_tot_return_gross_dvds(start=date_interval,end=as_of_date,per='d',currency='USD') * 
                               bq.data.cur_mkt_cap(start=date_interval,end=as_of_date,per='d',currency='USD',fill='prev')/1000000).group(grouping).sum()
    datafields['#Company'] = px_last.group(grouping).count()
    
    
    rename_col = {dates.format_for_query_string().upper():'Date',sector.format_for_query_string().upper():'Sector'}
#     univ = universe_picker.universe
#     if univ is None:
#         box_status.children=[]
#         return None
#     request=bql.Request(univ, datafields)
#     try:
#         response= bq.execute(request)
#     except bql.ResponseError as reserror:
#         universe_picker._update_status(reserror.exception_messages, warning=True)
#         return None
#     #response[0].df()
    def merge(response): return pd.concat([sir.df().rename(columns=rename_col).set_index(['Sector','Date']).dropna(how='all')[sir.name] for sir in response], axis=1)
#     df_main = merge(response).dropna()
    df_main = universe_picker.request_data(datafields, merge)
    if df_main is None:
        #box_status.children=[]
        return None
    df_main = df_main.dropna()
    df_main = df_main.loc[df_main.index.dropna()]
    df_main=df_main.reset_index()
    try: #2018-11-19 backward capable to BQL change
        df_main['Date']=df_main['Date'].apply(lambda x:dt.datetime.strptime(x[:10],'%Y-%m-%d'))
    except:
        pass
    date_max=df_main['Date'].max()
    date_min=df_main['Date'].min()
    
    # universe overall
    grouping = [dates]
    datafields = build_bq_dateitem(grouping, date_interval, as_of_date)
    datafields['Mkt Cap'] = (bq.data.cur_mkt_cap(start=date_interval,end=as_of_date,per='d',currency='USD',fill='prev')/1000000).group(grouping).sum()
    datafields['Return'] = (bq.data.day_to_day_tot_return_gross_dvds(start=date_interval,end=as_of_date,per='d',currency='USD') * 
                               bq.data.cur_mkt_cap(start=date_interval,end=as_of_date,per='d',currency='USD',fill='prev')/1000000).group(grouping).sum()
    datafields['#Company'] = px_last.group(grouping).count()
#     request=bql.Request(univ, datafields)
#     response= bq.execute(request)
    #response[0].df()
    def merge(response): return pd.concat([sir.df().rename(columns=rename_col).set_index(['Date']).dropna(how='all')[sir.name] for sir in response], axis=1)
    #df_overall = merge(response).dropna()
    df_overall = universe_picker.request_data(datafields, merge)
    if df_overall is None:
        #box_status.children=[]
        return None
    df_overall = df_overall.dropna()
    df_overall['Sector'] = "Overall Universe"
    df_overall=df_overall.reset_index()
    df_main = df_main.append(df_overall)[df_main.columns]
    
    df_main=df_main.set_index(['Sector','Date']).apply(lambda x:round(x*100))
    df_main['#Company']=df_main['#Company'].apply(lambda x:x/100)
    df_main['Price']=df_main['Return']/df_main['Mkt Cap']
    grid_data = df_main.xs(date_max,level='Date')
    
    df_return = df_main.reset_index().pivot(index='Date',columns='Sector',values='Price')
    df_return = df_return.dropna()
    #df_return = (df_return.pct_change().fillna(value=0)+1).cumprod()-1
    df_return = (df_return.fillna(value=0)+1).cumprod()-1
    
    def on_selected_change(*args):
        global grid_data
        try:
            if args[0]['old']['selected']['type'] == 'float':
                args[0]['old']['selected']['type'] = 'object'
        except:
            pass

        try:
            if args[0]['new']['selected']['type'] == 'float':
                args[0]['new']['selected']['type'] = 'object'
        except:
            pass
    

        if(indx_px_plot._figure.marks[0].selected is None or not indx_px_plot._figure.marks[0].selected):
            return
        #try:
        date=df_return.index[indx_px_plot._figure.marks[0].selected][0]
#             grid.data = df_main.xs(date,level='Date').reset_index()
        asof_datepicker.unobserve(on_datepicker)
        asof_datepicker.value = date.strftime('%Y-%m-%d')
        asof_datepicker.observe(on_datepicker)
        grid_data = df_main.xs(date,level='Date')
        update_heatmap(grid_data)
#         except:
#             pass
    display_fields=[]


    def on_datepicker(*args):
        intsel.unobserve(on_selected_change)
        intsel.selected = [np.datetime64(dt.datetime.strptime(asof_datepicker.value,'%Y-%m-%d'))]
        grid_data = df_main.xs(asof_datepicker.value,level='Date')
        update_heatmap(grid_data)
        intsel.observe(on_selected_change)
    
    asof_datepicker = bqw.DatePicker(date_format='%Y-%m-%d',description='As of date: ',value = date_max.strftime('%Y-%m-%d'), 
                                     min_date=date_min.strftime('%Y-%m-%d'), max_date=date_max.strftime('%Y-%m-%d'), layout={'left':'5px'})
    asof_datepicker.observe(on_datepicker)

    flag_indx = False
    if universe_picker._dropdown_type.value=='Index':
        flag_indx = True
        indx = universe_picker._ac_index.value
        datafields=OrderedDict()
        datafields['Price'] = px_last
        datafields['Return'] = (bq.data.day_to_day_tot_return_gross_dvds(start=date_interval,end=as_of_date,per='d',currency='USD')+1).cumprod()-1
        datafields['maxmin'] = mxmn.with_additional_parameters(period=20)
        request=bql.Request(indx, datafields)
        response= bq.execute(request)
        #df_indx_px = response[0].df().set_index('DATE')[['Price','4w High','4w Low']].dropna()
        def merge(response): return pd.concat([sir.df().set_index('DATE') for sir in response], axis=1)
        df_indx_px = merge(response).rename(columns={'MINLINE':'MinLine', 'MAXLINE':'MaxLine'}) #backward capable for BQL return header in capital letters
        #df_indx_px = df_indx_px[['Price','MinLine','MaxLine']].dropna()
        indx_px_plot = bqv.InteractiveLinePlot(df_indx_px[['Price','MinLine','MaxLine']].dropna(), title='Index Level and 4 weeks High Low- ' +indx, hide_controls=True)
        
    else:
        df_indx_px = df_return[['Overall Universe']]+1
        indx_px_plot = bqv.InteractiveLinePlot(df_indx_px[['Overall Universe']], title='Index Level (market cap weighted)', hide_controls=True)
        indx = universe_picker.description
        
    intsel = IndexSelector(scale=indx_px_plot._figure.axes[0].scale, marks=indx_px_plot._figure.marks)
    intsel.observe(on_selected_change)
    indx_px_plot._figure.interaction=intsel
    lst_inactive_intsel = []
    def on_grid_select(obj=None):

        global selected_sector
        #if obj.get('name')=='selected_row_indices':
        #selected_sector = grid.data['Sector'][obj.get('new')[0]]
        selected_sector = heatmap.get_selected_cells()[0][0]
        if selected_sector != 'Overall Universe':
            sector_return_chart.push(df_return[['Overall Universe',selected_sector]])
        else:
            sector_return_chart.push(df_return[[selected_sector]])
        sector_return_chart._figure.title = 'Return - ' + selected_sector
        sc_y2 = bqp.LinearScale()
        ax_y2 = bqp.Axis(scale=sc_y2, orientation='vertical',side='right',grid_lines='none')
        sector_return_chart.figure.axes=[sector_return_chart.figure.axes[0],sector_return_chart.figure.axes[1],ax_y2]

        adv_dec_plot.push(df_adv_dec[['Advanced','Declined']].loc[selected_sector])
        adv_dec_plot._figure.title = '% Movers - ' + selected_sector
        sc_x = adv_dec_plot.figure.axes[0].scale
        sc_y = adv_dec_plot.figure.axes[1].scale
        sc_y2 = bqp.LinearScale()
        ax_y2 = bqp.Axis(scale=sc_y2, orientation='vertical', label='Cum Sum',side='right',grid_lines='none')
        intsel = IndexSelector(scale=sc_x, marks=indx_px_plot._figure.marks)
        line = Lines(x=df_adv_dec.loc[selected_sector].index, y=df_adv_dec['Cum Sum'].loc[selected_sector],
                     scales={'x': sc_x, 'y': sc_y2},colors=['#F39F41','#626262'],title='% Movers - ' + selected_sector)

        adv_dec_plot.figure.marks=[adv_dec_plot.figure.marks[0]] + [line]
        adv_dec_plot.figure.axes=[adv_dec_plot.figure.axes[0],adv_dec_plot.figure.axes[1],ax_y2]
        df_MA = df_main[['>10D Avg','>50D Avg','>250D Avg']]
        MA_Line_plot.push(df_MA.loc[selected_sector])
        MA_Line_plot._figure.title = '% Above Moving Avg - ' + selected_sector
        MA_Line_plot = bqv.LinePlot(df_MA.loc[selected_sector],title='% Above Moving Avg - ' + selected_sector)
        sc_y2 = bqp.LinearScale()
        ax_y2 = bqp.Axis(scale=sc_y2, orientation='vertical',side='right',grid_lines='none')
        MA_Line_plot.figure.axes=[MA_Line_plot.figure.axes[0],MA_Line_plot.figure.axes[1],ax_y2]
        df_RSI = df_main[['RSI > 70']]
        df_RSI['RSI < 30'] = -df_main['RSI < 30']
        RSI_Line_plot.push(df_RSI.loc[selected_sector])
        RSI_Line_plot._figure.title = '% RSI Overbought vs Oversold - ' + selected_sector
        df_RSI['RSI Net'] = df_main['RSI > 70'] - df_main['RSI < 30']
        df_RSI['Cum RSI Net']=df_RSI['RSI Net'].groupby(level=[0]).cumsum()
        sc_x = RSI_Line_plot.figure.axes[0].scale
        sc_y = RSI_Line_plot.figure.axes[1].scale
        sc_y2 = bqp.LinearScale()
        ax_y2 = bqp.Axis(scale=sc_y2, orientation='vertical', label='Cum Sum',side='right')
        line = Lines(x=df_RSI.loc[selected_sector].index, y=df_RSI['Cum RSI Net'].loc[selected_sector],
                     scales={'x': sc_x, 'y': sc_y2},colors=['#F39F41','#626262'])
        RSI_Line_plot.figure.marks=[RSI_Line_plot.figure.marks[0]] + [line]
        RSI_Line_plot.figure.axes=[RSI_Line_plot.figure.axes[0],RSI_Line_plot.figure.axes[1],ax_y2]
        label_detail.value = apputils._HTML_TITLE.format('Selected group: ' + selected_sector)
        
    heatmap = heatMap.GenerateHeatMap()
    heatmap.generate_heatmaps([{'data': grid_data[['#Company']],'table_mode':True}]+
                              [{'data':grid_data[[name]],'reverse_color':True if name in lower_is_better_fields else False,
                                'upper_bound':upper_thurshold.get(name,upper_thurshold.get('default',None)),
                                'lower_bound':lower_thurshold.get(name,lower_thurshold.get('default',None)),
                                'tooltip': 'Highlighted {color} if {indicator} is over {value}. {low_bound_str}'.format(color='red' if name in lower_is_better_fields else 'green',
                                                                                        indicator = name,
                                                                                        value = upper_thurshold.get(name,upper_thurshold.get('default','NA')),
                                                                                        low_bound_str = 'Highlighted {color} if {indicator} is below {value}'.format(
                                                                                        color='green' if name in lower_is_better_fields else 'red',
                                                                                        indicator = name,
                                                                                        value = lower_thurshold.get(name,lower_thurshold.get('default','NA'))
                                                                                        ) if lower_thurshold.get(name,lower_thurshold.get('default',None)) is not None else ''
                                                                                        )
                               } for name in grid_data.columns if name not in ['#Company','Price','Mkt Cap','Return']], 
                          empty_space_width='1px',button_width='75px', header_height='30px',row_select=True,call_back=on_grid_select, upper_bound=upper_thurshold.get('default',None))
    
    grid=heatmap.get_widget()
    #ui_main = VBox([indx_px_plot.show(),label_asof, grid])
    label_indx_plot = HTML(apputils._HTML_STR_GRAY.format('Drag on the price chart to move back in time for the table below') ,layout={'left':'20px'})
    label_grid = HTML(apputils._HTML_STR_GRAY.format('Numbers in the table represent % of company in the group have met the technical analysis condition'),layout={'left':'20px'})
    ui_main = VBox([label_indx_plot,indx_px_plot.show(),HBox([asof_datepicker],layout={'margin':'0px 50px 0px 0px'}), label_grid, grid], layout={'overflow_x' : 'hidden'})
    #grid.observe(on_grid_select)
    
    selected_sector = df_main.index.levels[0][0]
    #df_return[indx]=df_indx_px['Return']#(df_indx_px['Price'].pct_change()+1).cumprod()-1
    df_return=df_return.dropna()
    
    df_adv_dec = df_main[['Advanced']]
    df_adv_dec['Declined']=-df_main['Declined']
    # Chart - Adv dec chart
    adv_dec_plot = bqv.BarPlot(df_adv_dec[['Advanced','Declined']].loc[selected_sector],bar_type='stacked',padding=0.01,
                               title='Cumulative # Movers - ' + selected_sector)
    adv_dec_plot.figure.animation_duration=1000
    sc_x = adv_dec_plot.figure.axes[0].scale
    sc_y = adv_dec_plot.figure.axes[1].scale
    sc_y2 = bqp.LinearScale()
    ax_y2 = bqp.Axis(scale=sc_y2, orientation='vertical', label='Cum Sum',side='right')
    df_adv_dec['Net'] = df_adv_dec['Advanced'] + df_adv_dec['Declined']
    df_adv_dec['5D avg'] = df_adv_dec['Net'].rolling(5).mean()
    df_adv_dec['Cum Sum']=df_adv_dec.groupby(level=0)['Net'].cumsum()

    line = Lines(x=df_adv_dec.loc[selected_sector].index, y=df_adv_dec['Cum Sum'].loc[selected_sector],
                 scales={'x': sc_x, 'y': sc_y2},colors=['#F39F41','#626262'])
    adv_dec_plot.figure.marks=[adv_dec_plot.figure.marks[0]] + [line]
    adv_dec_plot.figure.axes=[adv_dec_plot.figure.axes[0],adv_dec_plot.figure.axes[1],ax_y2]
    
    
    def _zoom_chart_onclick(btn):
        if btn.figure.height =='300px':
            btn.figure.height = '700px'
        else:
            btn.figure.height = '300px'
    btn_zoom_adv_dec = apputils.ButtonConfirm(_zoom_chart_onclick, req_confirm=False, icon = 'search', layout={'width':'50px'})
    btn_zoom_adv_dec.figure=adv_dec_plot
    
    df_MA = df_main[['>10D Avg','>50D Avg','>250D Avg']]
    MA_Line_plot = bqv.LinePlot(df_MA.loc[selected_sector],title='% Above Moving Avg - ' + selected_sector)
    sc_y2 = bqp.LinearScale()
    ax_y2 = bqp.Axis(scale=sc_y2, orientation='vertical',side='right',grid_lines='none')
    MA_Line_plot.figure.axes=[MA_Line_plot.figure.axes[0],MA_Line_plot.figure.axes[1],ax_y2]
    btn_zoom_MA_plot = apputils.ButtonConfirm(_zoom_chart_onclick, icon = 'search',req_confirm=False, layout={'width':'50px'})
    btn_zoom_MA_plot.figure = MA_Line_plot
    
    df_RSI = df_main[['RSI > 70']]
    df_RSI['RSI < 30'] = -df_main['RSI < 30']
    RSI_Line_plot= bqv.BarPlot(df_RSI.loc[selected_sector], bar_type='stacked',padding=0.01,title='% RSI Overbought vs Oversold - ' + selected_sector)
    RSI_Line_plot.figure.animation_duration=1000
    df_RSI['RSI Net'] = df_main['RSI > 70'] - df_main['RSI < 30']
    df_RSI['Cum RSI Net']=df_RSI.groupby(level=[0])['RSI Net'].cumsum()
    sc_x = RSI_Line_plot.figure.axes[0].scale
    sc_y = RSI_Line_plot.figure.axes[1].scale
    sc_y2 = bqp.LinearScale()
    ax_y2 = bqp.Axis(scale=sc_y2, orientation='vertical', label='Cum Sum',side='right',grid_lines='none')
    line = Lines(x=df_RSI.loc[selected_sector].index, y=df_RSI['Cum RSI Net'].loc[selected_sector],
                 scales={'x': sc_x, 'y': sc_y2},colors=['#F39F41','#626262'])
    RSI_Line_plot.figure.marks=[RSI_Line_plot.figure.marks[0]] + [line]
    RSI_Line_plot.figure.axes=[RSI_Line_plot.figure.axes[0],RSI_Line_plot.figure.axes[1],ax_y2]
    btn_zoom_RSI_plot = apputils.ButtonConfirm(_zoom_chart_onclick, icon = 'search',req_confirm=False, layout={'width':'50px'})
    btn_zoom_RSI_plot.figure = RSI_Line_plot
    
    sector_return_chart = bqv.LinePlot(df_return[['Overall Universe',selected_sector]],title='Cumulative Return - ' + selected_sector)
    sc_y2 = bqp.LinearScale()
    ax_y2 = bqp.Axis(scale=sc_y2, orientation='vertical',side='right',grid_lines='none')
    sector_return_chart.figure.axes=[sector_return_chart.figure.axes[0],sector_return_chart.figure.axes[1],ax_y2]
    btn_zoom_sector_return = apputils.ButtonConfirm(_zoom_chart_onclick,icon = 'search',req_confirm=False, layout={'width':'50px'})
    btn_zoom_sector_return.figure = sector_return_chart
    
    lst_ui_sector_chart = [HBox([btn_zoom_sector_return.show(), sector_return_chart.show()]),
                           HBox([btn_zoom_adv_dec.show(), adv_dec_plot.show()]),
                            HBox([btn_zoom_MA_plot.show(),MA_Line_plot.show()]),
                           HBox([btn_zoom_RSI_plot.show(),RSI_Line_plot.show()]
                                   )]
    label_sector_chart = Label('Select Group from the above heatmap to chart')
        
    ui_sector_chart = VBox([label_sector_chart] + lst_ui_sector_chart)
    
    
    label_detail = HTML(apputils._HTML_TITLE.format('Selected group: ' + selected_sector))
    btn_get_detail = apputils.ButtonConfirm(_get_sector_details, description = 'Get Details',req_confirm=False).show()
    box_heatmap_details = VBox(layout={'min_height':'1000px','top':'20px','bottom':'20px','overflow_y':'scroll'})
    box_details_contrils = HBox([label_detail, btn_get_detail])
    
    # Multi charts
    def _on_filter_selection(selection=None):
        rsi = df_RSI[['Cum RSI Net']]
        rsi = rsi.reset_index()
        rsi = rsi.pivot(index='Date',values='Cum RSI Net',columns='Sector')
        rsi_chart_multi.push(rsi[selection])

        return_chart_multi.push(df_return[selection])

        adv_dec = df_adv_dec[['Cum Sum']]
        adv_dec = adv_dec.reset_index()
        adv_dec = adv_dec.pivot(index='Date',values='Cum Sum',columns='Sector')
        adv_dec_chart_multi.push(adv_dec[selection])

        ma_short = df_MA[['>10D Avg']]
        ma_short = ma_short.reset_index()
        ma_short = ma_short.pivot(index='Date',values='>10D Avg',columns='Sector')
        ma_short_chart_multi.push(ma_short[selection])

        ma_mid = df_MA[['>50D Avg']]
        ma_mid  = ma_mid.reset_index()
        ma_mid  = ma_mid.pivot(index='Date',values='>50D Avg',columns='Sector')
        ma_mid_chart_multi.push(ma_mid[selection])


        ma_long = df_MA[['>250D Avg']]
        ma_long  = ma_long.reset_index()
        ma_long  = ma_long.pivot(index='Date',values='>250D Avg',columns='Sector')
        ma_long_chart_multi.push(ma_long[selection])
    
    rsi = df_RSI[['Cum RSI Net']]
    rsi = rsi.reset_index()
    rsi = rsi.pivot(index='Date',values='Cum RSI Net',columns='Sector')

    rsi_chart_multi = bqv.LinePlot(title='Cumulative %RSI Overbrought/Oversold')
    return_chart_multi = bqv.LinePlot(title='Cumulative Return')
    adv_dec_chart_multi = bqv.LinePlot(title= 'Cumulative Advance-Decline')
    ma_short_chart_multi = bqv.LinePlot(title ='>10days Moving Average')
    ma_mid_chart_multi = bqv.LinePlot(title= '>50days Moving Average')
    ma_long_chart_multi = bqv.LinePlot(title= '>250days Moving Average')
    box_multi_charts = VBox([return_chart_multi.show(), 
                             adv_dec_chart_multi.show(),
                             rsi_chart_multi.show(),
                             ma_short_chart_multi.show(),
                             ma_mid_chart_multi.show(),
                             ma_long_chart_multi.show()])
    filter_picker = apputils.FilterPicker(list(rsi.columns), on_change=_on_filter_selection,toggle_all=False)
    ui_multi_charts = VBox([filter_picker.show(),box_multi_charts])
    
    ui_details = VBox([box_details_contrils, box_heatmap_details], layout={'overflow_y':'hidden'})
    ui_output = Tab([ui_sector_chart,ui_multi_charts,ui_details],layout={'top':'10px'})
    ui_output.set_title(0,'Single Group Charts')
    ui_output.set_title(1,'Muti Group Charts')
    ui_output.set_title(2,'Detail Table')
    ui.children = [ui_control,ui_main,ui_output]
    #box_status.children=[]
    save_settings()
def update_heatmap(grid_data):
#     lst_heatmap = [{'data': grid_data[['#Company']],'table_mode':True}]
#     for name in grid_data.columns if name not in ['#Company','Price']:
        
    
    heatmap.generate_heatmaps([{'data': grid_data[['#Company']],'table_mode':True}]+
                              [{'data': grid_data[[name]],'reverse_color':True if name in lower_is_better_fields else False,
                                'upper_bound':upper_thurshold.get(name,upper_thurshold.get('default',None)),
                                'lower_bound':lower_thurshold.get(name,lower_thurshold.get('default',None)),
                                'tooltip': 'Highlighted {color} if {indicator} is over {value}. {low_bound_str}'.format(color='red' if name in lower_is_better_fields else 'green',
                                                                                                        indicator = name,
                                                                                                        value = upper_thurshold.get(name,upper_thurshold.get('default','NA')),
                                                                                                        low_bound_str = 'Highlighted {color} if {indicator} is Below {value}'.format(
                                                                                                        color='green' if name in lower_is_better_fields else 'red',
                                                                                                        indicator = name,
                                                                                                        value = lower_thurshold.get(name,lower_thurshold.get('default','NA'))
                                                                                                        ) if lower_thurshold.get(name,lower_thurshold.get('default',None)) is not None else ''
                                                                                                        )
                               } for name in grid_data.columns if name not in ['#Company','Price','Mkt Cap','Return']], 
                          quick_update=True, upper_bound=upper_thurshold.get('default',None))

# Market Breadth Technical Analysis
---

In [10]:
button_run = apputils.ButtonConfirm(procedure=button_run_on_click,description='Run', button_style='info',req_confirm=False).show()
univ_accordion = Accordion(children=[universe_picker.show()], layout=Layout(width='40%'))
univ_accordion.set_title(0, 'Universe')
date_accordion = Accordion(children=[date_picker.show()], layout=Layout(width='40%'))
date_accordion.set_title(0, 'Settings')
box_status = HBox([],layout=Layout(width='650px'))
ui_control=VBox([HBox([univ_accordion,date_accordion]),HBox([button_run,box_status])])
ui = VBox([ui_control],layout={'overflow_y':'hidden'})
if True:
    button_run_on_click()
preload_box.children=[]
ui

HTML(value='<style>.go-btn{font-family:Calibri!important;font-size:14px!important;border-radius:5px!important;…

In [11]:
import ipywidgets as ipw
from IPython.display import display
btn_css = ('<style>'
            '.go-btn{font-family:Calibri!important;'
            'font-size:13px!important;'
            'border-radius:1px!important;'
            'font-weight: bold!important;'
            'color:black!important;}'

            '.go-btn:active{background-color:transparent!important;}'
            '</style>'
            )
        
display(ipw.HTML(btn_css))

HTML(value='<style>.go-btn{font-family:Calibri!important;font-size:13px!important;border-radius:1px!important;…

In [12]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999

<IPython.core.display.Javascript object>